In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import style 
import cv2
import pickle
import time

style.use("ggplot")

size = 10
hm_episodes = 25000
move_penalty = 1
enemy_penalty = 300
food_reward = 25
epsilon = 0.9
eps_decay = 0.9998
show_every = 3000

start_q_table = None

learning_rate = 0.1
discount = 0.95

player_n = 1
food_n = 2
enemy_n = 3

d = {1:(155,175,0),
     2:(0,255,0),
     3:(0,0,255)}

class Blob:
    def __init__(self):
        self.x = np.random.randint(0,size)
        self.y = np.random.randint(0,size)
    def __str__(self):
        return f"{self.x},{self.y}"
    
    def __sub__(self,other):
        return (self.x-other.x,self.y-other.y)
    
    def action(self,choice):
        if choice==0:
            self.move(x=1,y=1)
        if choice==1:
            self.move(x=-1,y=-1)
        if choice==2:
            self.move(x=-1,y=1)
        if choice==0:
            self.move(x=1,y=-1)
            
    def move(self,x=False,y=False):
        if not x:
            self.x+=np.random.randint(-1,2)
        else:
            self.x+=x
        if not y:
            self.y+=np.random.randint(-1,2)
        else:
            self.y+=y
            
        if self.x<0:
            self.x=0
        elif self.x>size-1:
            self.x = size-1
        if self.y<0:
            self.y=0
        elif self.y>size-1:
            self.y = size-1
if start_q_table is None:
    q_table = {}
    for x1 in range(-size+1,size):
        for y1 in range(-size+1,size):
            for x2 in range(-size+1,size):
                for y2 in range(-size+1,size):
                    q_table[(x1,y1),(x2,y2)]=[np.random.uniform(-5,0) for i in range(4)]
else:
    with open(start_q_table,"rb") as f:
        q_table = pickle.load(f)
        
episode_rewards = []
for episode in range(hm_episodes):
    player = Blob()
    food = Blob()
    enemy = Blob()
    
    if episode % show_every==0:
        print(f"on # {episode},epsilon:{epsilon}")
        print(f"{show_every} ep mean {np.mean(episode_rewards[-show_every:])}")
        show = True
    else:
        show = False
        
    episode_reward  = 0
    for i in range(200):
        obs = (player-food,player-enemy)
        if np.random.random()>epsilon:
            action = np.argmax(q_table[obs])
        else:
            action = np.random.randint(0,4)
            
        player.action(action)
        
        if player.x==enemy.x and player.y==enemy.y:
            reward = -enemy_penalty
        elif player.x==food.x and player.y==food.y:
            reward = food_reward
        else:
            reward = -move_penalty
            
        new_obs = (player-food,player-enemy)
        max_future_q = np.max(q_table[new_obs])
        current_q = q_table[obs][action]
        
        if reward == food_reward:
            new_q = food_reward
        elif reward == -enemy_penalty:
            new_q = -enemy_penalty
        else:
            new_q = (1-learning_rate)*current_q+learning_rate*(reward+discount*max_future_q)
        q_table[obs][action]=new_q
        
        if show:
            env = np.zeros((size,size,3),dtype=np.uint8)
            env[food.x][food.y]=d[food_n]
            env[player.x][player.y]=d[player_n]
            env[enemy.x][enemy.y]=d[enemy_n]
            
            img = Image.fromarray(env,"RGB")
            img = img.resize((300,300))
            cv2.imshow("",np.array(img))
            if reward==food_reward or reward==-enemy_penalty:
                if cv2.waitKey(500) & 0xFF == ord('q'):
                    break
        episode_reward+=reward
        if reward==food_reward or reward==-enemy_penalty:
            break
    episode_rewards.append(episode_reward)
    epsilon *= eps_decay

moving_avg = np.convolve(episode_rewards,np.ones((show_every,))/show_every,mode='valid')
plt.plot([i for i in range(len(moving_avg))],moving_avg)
plt.ylabel(f"rward (show_every)ma")
plt.xlabel(f"episode #")
plt.show()

with open(f"qtable-{int(time.time())}.pickle","wb") as f:
    pickle.dump(q_table)

            

on # 0,epsilon:0.9
3000 ep mean nan


C:\Users\HP\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\HP\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


on # 3000,epsilon:0.49390083359356435
3000 ep mean -175.82433333333333
on # 6000,epsilon:0.27104225936046566
3000 ep mean -130.95666666666668
on # 9000,epsilon:0.14874221981913022
3000 ep mean -104.13133333333333
on # 12000,epsilon:0.08162656262136181
3000 ep mean -87.08466666666666
on # 15000,epsilon:0.044794919246742226
3000 ep mean -73.85566666666666
on # 18000,epsilon:0.024582497729691496
3000 ep mean -66.284
on # 21000,epsilon:0.01349035124500733
3000 ep mean -58.263
on # 24000,epsilon:0.0074032174726434705
3000 ep mean -51.065666666666665


<Figure size 640x480 with 1 Axes>

TypeError: dump() missing required argument 'file' (pos 2)